# Anvil

Disponibilizar como web app

In [9]:
!pip install anvil-uplink
import anvil.server
 
anvil.server.connect("3KI3P5YQ6A5PZ3XJ7YTRTOJH-L5LICJ5ST42K3553")

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


# Extrair as informações desejadas.

Guarda em um dicionario de dicionarios.

**Dia_da_Semana: Disciplina: [[inicio,fim],...    ]**



Dia_da_Semana: dicionario


Disciplina: valor de dia_da_semana e também um dicionario


[[inicio,fim],...    ]: Lista de listas, valor do dicionario disciplina, tendo o inicio e o fim do horario para cada linha

In [10]:
string = '''SEGUNDA-FEIRA
15:10 - 16:00	22000269 - M1 - SISTEMAS DIGITAIS AVANÇADOS	Não informado
16:00 - 16:50	22000269 - M1 - SISTEMAS DIGITAIS AVANÇADOS	Não informado
TERÇA-FEIRA
15:10 - 16:00	22000270 - T1 - SISTEMAS OPERACIONAIS	Não informado
16:00 - 16:50	22000270 - T1 - SISTEMAS OPERACIONAIS	Não informado
QUARTA-FEIRA
15:10 - 16:00	22000236 - T1 - ANÁLISE DE CIRCUITOS I	Não informado
16:00 - 16:50	22000236 - T1 - ANÁLISE DE CIRCUITOS I	Não informado
SEXTA-FEIRA
10:00 - 10:50	22000237 - M1 - PROGRAMAÇÃO DE SISTEMAS	Não informado
10:50 - 11:40	22000237 - M1 - PROGRAMAÇÃO DE SISTEMAS	Não informado
15:10 - 16:00	22000189 - T1 - ENGENHARIA DE SOFTWARE I	Não informado
16:00 - 16:50	22000189 - T1 - ENGENHARIA DE SOFTWARE I	Não informado'''

In [11]:
import re 
from collections import defaultdict

@anvil.server.callable
def criar_dicionario(string):
  linhas = string.split(sep = '\n')

  dicionario = defaultdict(lambda : [])


  #EXTRAIR OS DIAS 

  for x in range(len(linhas) -1):
    
    extraido = re.findall(r'^[^\d]*$',linhas[x])  #CHECA SE NÃO TEM NENHUM NÚMERO, CASO NAO TENHA, É UM DIA DA SEMANA
    if len(extraido)>0:
      dia = extraido
      linha_valida = True
      x = x+1
      disciplina_dict = defaultdict(lambda : [])

      while( linha_valida  and x< len(linhas)):
        if(len(re.findall(r'^[^\d]*$',linhas[x])) ==0 ):
          
          #EXTRAIR O NOME DA DISCIPLINA
          
          aux = linhas[x].split(sep = '-')[-1]
          aux = re.findall(r'(.*)\sNão informado',aux)
          
          disciplina = aux[0].strip()
          

          #EXTRAIR AS HORAS
          
          horas = re.findall(r'\d{2}:\d{2}',linhas[x])
          
          
          x = x+1
          disciplina_dict[disciplina].append(horas)
          
        else: 
          

          linha_valida = False
      
      dicionario[dia[0]] = dict(disciplina_dict)
      disciplina_dict = defaultdict(lambda : [])
  return dict(dicionario)

#dicionario = criar_dicionario(string)  
#print(dicionario)

    

    




# Ajustar o dicionario de dicionarios:

## Converter os dias da semana em números:

In [12]:
#Converter os dias da semana em números

@anvil.server.callable
def rename_keys(d, keys):
    return dict([(keys.get(k), v) for k, v in d.items()])

conversao = {'SEGUNDA-FEIRA': 0, 'TERÇA-FEIRA':1, 'QUARTA-FEIRA':2,'QUINTA-FEIRA':3,'SEXTA-FEIRA':4,'SÁBADO':5,'DOMINGO':6, 'SABADO' : 5}

#dicionario = rename_keys(dicionario,conversao)


## Simplificar os horários :transformar horarios como ['15:10', '16:00'], ['16:00', '16:50'] -> Em apenas  [15:10,16:50]

In [13]:


# Transformar horarios como ['15:10', '16:00'], ['16:00', '16:50'] -> Em apenas  [15:10,16:50]

from dateutil.rrule import rrule, WEEKLY, MO
from datetime import date


def ajustar(lista):
  lista2 = list()
  aux = list()

  if len(lista) == 1:
    return [(lista[0][0],lista[0][1])]

  for x in range(len(lista) -1):
    passou = False
    fim = lista[x][1]
    for y in range (x+1,len(lista)):
      if fim == lista[y][0]:
        passou = True
        fim = lista[y][1]
        
    if passou and fim not in aux:
      lista2.append( (lista[x][0],fim))
      aux.append(fim)
    else:
      if fim not in aux:
        lista2.append((lista[x][0], fim))

  return lista2 




# Criar calendario do 0 com icalendar


In [14]:
!pip install icalendar

In [15]:
#CALCULAR O PRÓXIMO DIA

from dateutil.rrule import rrule, WEEKLY, MO
from datetime import date

next_monday = rrule(freq=WEEKLY, dtstart=date.today(), byweekday=6, count=1)[0]
#next_monday.hour

In [16]:
from datetime import datetime, timedelta
from icalendar import Calendar, Event,Alarm
import pytz


def criar_evento(cal,nome,inicio, fim): #comeco:datetime
                                        # fim: datetime

  event = Event()
  event.add('summary', nome)

  tzone = pytz.timezone('America/Sao_Paulo')

  event.add('dtstart', tzone.localize(inicio, is_dst=None))
  event.add('dtend', tzone.localize(fim, is_dst=None))
  event.add('rrule', {'freq': ['WEEKLY']})

  alarm = Alarm()
  alarm.add('action', 'DISPLAY')
  alarm.add('trigger', timedelta(minutes=-10))

  event.add_component(alarm)

  cal.add_component(event)


inicio = [2021,8,5,10,5]
fim = [2021,8,5,10,35]

@anvil.server.callable
def criar_ical(dicionario): 
  dicionario = rename_keys(dicionario,conversao)
  cal = Calendar()

  cal.add('prodid', '-//Google Inc//Google Calendar 70.9054//EN')
  cal.add('version', '2.0')
  

  #---------------------- 
  for k,v in dicionario.items():
    for key, value in v.items():
      next_day = rrule(freq=WEEKLY, dtstart=date.today(), byweekday=k, count=1)[0]
      time = datetime.strptime(ajustar(value)[0][0], '%H:%M')
      end_time = datetime.strptime(ajustar(value)[0][1], '%H:%M')
      
      comeco = datetime(next_day.year,next_day.month,next_day.day, time.hour, time.minute)
      fim = datetime(next_day.year,next_day.month,next_day.day, end_time.hour, end_time.minute)

      criar_evento(cal,key,comeco, fim)
  #--------------------------



  with open('calendario_2021_2.ics', 'wb') as fw:
    fw.write(cal.to_ical())

  import anvil.media
  media_obj = anvil.media.from_file('calendario_2021_2.ics')
  return media_obj



#criar_ical(dicionario) 





In [ ]:
anvil.server.wait_forever()